In [1]:
import pandas as pd
from datetime import date,datetime

df1 = pd.read_pickle('./input_tables/pickle/daily_dlqt/daily_delqt_short_1_0.pkl')
df2 = pd.read_pickle('./input_tables/pickle/daily_dlqt/daily_delqt_short_2_0.pkl')
df3 = pd.read_pickle('./input_tables/pickle/daily_dlqt/daily_delqt_short_3_0.pkl')
df4 = pd.read_pickle('./input_tables/pickle/daily_dlqt/daily_delqt_short_4_0.pkl')
df5 = pd.read_pickle('./input_tables/pickle/daily_dlqt/daily_delqt_short_5_0.pkl')

In [2]:
daily_dlqt = pd.concat([df1, df2, df3, df4, df5], ignore_index = True)

In [3]:
# 날짜형식 데이터 Date 형식으로 변환
daily_dlqt['DLYSTADAT'] =   pd.to_datetime(daily_dlqt['DLYSTADAT'].astype(str), format='%Y%m%d') 
daily_dlqt['PMTAFTDAT'] =   pd.to_datetime(daily_dlqt['PMTAFTDAT'].astype(str), format='%Y%m%d') 
daily_dlqt['ACPPMTDAT'] =   pd.to_datetime(daily_dlqt['ACPPMTDAT'].astype(str), format='%Y%m%d')

# 실납부일과 기준일자 사이의 날짜 구하기
daily_dlqt['ACT_DLQT_DAY'] = (daily_dlqt['DLYSTADAT']-daily_dlqt['ACPPMTDAT']).dt.days

# index 지우기 
daily_dlqt.drop(['index'], axis=1, inplace=True)

# 정렬하기
daily_dlqt = daily_dlqt.sort_values(['EXCFLDNBR', 'DLYSTADAT', 'RFDFLDODR'], ascending = [True, True, False])

In [4]:
daily_dlqt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468574 entries, 3116 to 467043
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DLYSTADAT     468574 non-null  datetime64[ns]
 1   EXCFLDNBR     468574 non-null  object        
 2   PAPCUSCDE     468574 non-null  int64         
 3   SCHFLDKND     468574 non-null  object        
 4   SCHSEQNBR     468574 non-null  int64         
 5   RFDFLDODR     468574 non-null  int64         
 6   PMTAFTDAT     468574 non-null  datetime64[ns]
 7   ACPPMTDAT     468574 non-null  datetime64[ns]
 8   DLYTOTDAY     468574 non-null  int64         
 9   UNCPRNAMT     468574 non-null  int64         
 10  DLYPRNAMT     468574 non-null  int64         
 11  DLYINTAMT     468574 non-null  int64         
 12  DLYETCAMT     468574 non-null  int64         
 13  DLYLESAMT     468574 non-null  int64         
 14  CLNINTAMT     468574 non-null  int64         
 15  ACT_DLQT_DAY  

In [ ]:
* 마스터성(최근 조회기준일자 기준) 자료 Grouping
    - A : 실행번호별 1일,3일, 5일, 10일, 15일, 20일, 30일, 60일, 90일 연체 최초 발생일자, 발생월
    - B : 실행번호별 1일,3일, 5일, 10일, 15일, 20일, 30일, 60일, 90일 연체 최초 발생회차(원리금스케줄)
    - C : 실행번호별 연체경험횟수
    - D : 실행번호별 최대연체일수
    - E : 실행번호별 회차별 연체평균일수(연체해제평균일자)
    
* 시계열(월말 기준) 자료
  - F : 월말별 실행번호별 5일, 10일, 20일, 30일, 60일, 90일 이상 건수, 연체원리금


In [5]:
# 연체일수 Label 함수

def dlqt_class(day) : 
    if day >= 5 :
        return '5D'
    elif day >= 10 :
        return '10D'
    elif day >= 15 :
        return '15D'
    elif day >= 20 :
        return '20D'
    elif day >= 30 :
        return '30D'
    elif day >= 60 :
        return '60D'
    elif day >= 90 :
        return '90D'
    else :
        return 'U5'

In [8]:
daily_dlqt['DLQT_CLASS'] = daily_dlqt['ACT_DLQT_DAY'].apply(lambda x : dlqt_class(x))

In [116]:
daily_dlqt

,DLYSTADAT,EXCFLDNBR,PAPCUSCDE,SCHFLDKND,SCHSEQNBR,RFDFLDODR,PMTAFTDAT,ACPPMTDAT,DLYTOTDAY,UNCPRNAMT,DLYPRNAMT,DLYINTAMT,DLYETCAMT,DLYLESAMT,CLNINTAMT,ACT_DLQT_DAY,DLQT_CLASS
3116,2017-09-16,216AL1000010001,0,FI670001,10,9,2017-09-15,2017-09-15,1,48147742,743099,207901,0,625,0,1,U5
3313,2017-09-17,216AL1000010001,0,FI670001,10,9,2017-09-15,2017-09-15,2,48147742,743099,207901,0,625,0,2,U5
5620,2017-12-16,216AL1000010001,0,FI670001,13,12,2017-12-15,2017-12-15,1,45890293,757204,193796,0,625,0,1,U5
5919,2017-12-17,216AL1000010001,0,FI670001,13,12,2017-12-15,2017-12-15,2,45890293,757204,193796,0,625,0,2,U5
8252,2018-02-16,216AL1000010001,0,FI670001,15,14,2018-02-15,2018-02-15,1,44361572,766756,184244,0,625,0,1,U5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467041,2021-06-27,221LO0056140001,0,FI670001,2,1,2021-06-25,2021-06-25,2,19497619,502381,59616,0,150,0,2,U5
459381,2021-06-26,221LO0056200001,0,FI670001,2,1,2021-06-25,2021-06-25,1,15000000,0,30205,0,6,0,1,U5
467042,2021-06-27,221LO0056200001,0,FI670001,2,1,2021-06-25,2021-06-25,2,15000000,0,30205,0,6,0,2,U5
459382,2021-06-26,221LO0056380001,0,FI670001,2,1,2021-06-25,2021-06-25,1,39000000,0,78534,0,16,0,1,U5


In [10]:
# 연체일수 Threshold Hit 기준 DataFrame 생성
# 1. 연체 1일인 연체내역
dlqt_1D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 1].reset_index()

# 2. 연체 3일인 연체내역
dlqt_3D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 3].reset_index()

# 3. 연체 5일인 연체내역
dlqt_5D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 5].reset_index()

# 4. 연체 10일인 연체내역
dlqt_10D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 10].reset_index()

# 5. 연체 15일인 연체내역
dlqt_15D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 15].reset_index()

# 6. 연체 20일인 연체내역
dlqt_20D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 20].reset_index()

# 7. 연체 30일인 연체내역
dlqt_30D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 30].reset_index()

# 8. 연체 60일인 연체내역
dlqt_60D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 60].reset_index()

# 9. 연체 90일인 연체내역
dlqt_90D = daily_dlqt[daily_dlqt['ACT_DLQT_DAY'] == 90].reset_index()

In [18]:
# 실행번호별 연체발생 함수(1~90일 최초 발생일자, 발생월 Dataframe 생성)

def get_dlqt_start_date(df) :
    filtered_df = df.groupby(['EXCFLDNBR','DLYSTADAT'])['RFDFLDODR'].min().reset_index()
    return filtered_df.groupby(['EXCFLDNBR'])['DLYSTADAT'].min().reset_index()


In [19]:
# 연체경험 일자별 시작일자 Dataframe 만들기
dlqt_1D_experience_df = get_dlqt_start_date(dlqt_1D)
dlqt_3D_experience_df = get_dlqt_start_date(dlqt_3D)
dlqt_5D_experience_df = get_dlqt_start_date(dlqt_5D)
dlqt_10D_experience_df = get_dlqt_start_date(dlqt_10D)
dlqt_15D_experience_df = get_dlqt_start_date(dlqt_15D)
dlqt_20D_experience_df = get_dlqt_start_date(dlqt_20D)
dlqt_30D_experience_df = get_dlqt_start_date(dlqt_30D)
dlqt_60D_experience_df = get_dlqt_start_date(dlqt_60D)
dlqt_90D_experience_df = get_dlqt_start_date(dlqt_90D)

In [21]:
dlqt_5D_experience_df

,EXCFLDNBR,DLYSTADAT
0,216AL1000040001,2020-08-23
1,216AL1000060001,2017-07-22
2,216AL1000070001,2018-09-26
3,216AL1000120001,2018-09-26
4,216AL1000200001,2018-09-26
...,...,...
3518,221LO0043040001,2021-06-30
3519,221LO0044060001,2021-06-20
3520,221LO0046820001,2021-06-30
3521,221LO0048030001,2021-06-30


In [26]:
# 실행번호별 연체발생 회차 함수(1~90일 최초 발생일자, 발생월 Dataframe 생성)

def get_dlqt_start_seq(df) :
    filtered_df = df.groupby(['EXCFLDNBR'])['RFDFLDODR'].min().reset_index()
    return filtered_df

In [27]:
# 연체경험 일자별 시작회차 Dataframe 만들기
dlqt_1D_experience_seq_df = get_dlqt_start_seq(dlqt_1D)
dlqt_3D_experience_seq_df = get_dlqt_start_seq(dlqt_3D)
dlqt_5D_experience_seq_df = get_dlqt_start_seq(dlqt_5D)
dlqt_10D_experience_seq_df = get_dlqt_start_seq(dlqt_10D)
dlqt_15D_experience_seq_df = get_dlqt_start_seq(dlqt_15D)
dlqt_20D_experience_seq_df = get_dlqt_start_seq(dlqt_20D)
dlqt_30D_experience_seq_df = get_dlqt_start_seq(dlqt_30D)
dlqt_60D_experience_seq_df = get_dlqt_start_seq(dlqt_60D)
dlqt_90D_experience_seq_df = get_dlqt_start_seq(dlqt_90D)

In [28]:
dlqt_1D_experience_seq_df

,EXCFLDNBR,RFDFLDODR
0,216AL1000010001,9
1,216AL1000030001,9
2,216AL1000040001,9
3,216AL1000050001,8
4,216AL1000060001,7
...,...,...
27860,221LO0053780001,1
27861,221LO0054020001,1
27862,221LO0056140001,1
27863,221LO0056200001,1


In [36]:
#실행번호별 연체경험 횟수
# 1. 연체 1일 경험횟수 (회차별 연체 1일 이상인 회차의 갯수)
dlqt_1D_experience_times_temp = dlqt_1D.groupby(['EXCFLDNBR','RFDFLDODR'])['ACT_DLQT_DAY'].count().reset_index()
dlqt_1D_experience_times_df = dlqt_1D_experience_times_temp.groupby(['EXCFLDNBR'])['RFDFLDODR'].count().reset_index()

# 2. 연체 3일 경험횟수 (회차별 연체 3일 경험 회차의 갯수)
dlqt_3D_experience_times_temp = dlqt_3D.groupby(['EXCFLDNBR','RFDFLDODR'])['ACT_DLQT_DAY'].count().reset_index()
dlqt_3D_experience_times_df = dlqt_3D_experience_times_temp.groupby(['EXCFLDNBR'])['RFDFLDODR'].count().reset_index()

# 5. 연체 5일 경험횟수 (회차별 연체 5일 경험 회차의 갯수)
dlqt_5D_experience_times_temp = dlqt_5D.groupby(['EXCFLDNBR','RFDFLDODR'])['ACT_DLQT_DAY'].count().reset_index()
dlqt_5D_experience_times_df = dlqt_5D_experience_times_temp.groupby(['EXCFLDNBR'])['RFDFLDODR'].count().reset_index()

In [37]:
dlqt_5D_experience_times_df

,EXCFLDNBR,RFDFLDODR
0,216AL1000040001,1
1,216AL1000060001,5
2,216AL1000070001,1
3,216AL1000120001,1
4,216AL1000200001,1
...,...,...
3518,221LO0043040001,1
3519,221LO0044060001,1
3520,221LO0046820001,1
3521,221LO0048030001,1


In [39]:
# 실행번호별 최대연체일수

dlqt_experience_max_days = daily_dlqt.groupby(['EXCFLDNBR'])['ACT_DLQT_DAY'].max().reset_index()

In [40]:
dlqt_experience_max_days

,EXCFLDNBR,ACT_DLQT_DAY
0,216AL1000010001,4
1,216AL1000030001,4
2,216AL1000040001,5
3,216AL1000050001,2
4,216AL1000060001,9
...,...,...
27921,221LO0053780001,2
27922,221LO0054020001,2
27923,221LO0056140001,2
27924,221LO0056200001,2


In [42]:
# 실행번호별 평균연체일수

dlqt_experience_average_days_temp = daily_dlqt.groupby(['EXCFLDNBR', 'RFDFLDODR'])['ACT_DLQT_DAY'].max().reset_index()
dlqt_experience_average_days = dlqt_experience_average_days_temp.groupby(['EXCFLDNBR'])['ACT_DLQT_DAY'].mean().reset_index()

In [43]:
dlqt_experience_average_days

,EXCFLDNBR,ACT_DLQT_DAY
0,216AL1000010001,2.285714
1,216AL1000030001,2.500000
2,216AL1000040001,2.636364
3,216AL1000050001,2.000000
4,216AL1000060001,3.555556
...,...,...
27921,221LO0053780001,2.000000
27922,221LO0054020001,2.000000
27923,221LO0056140001,2.000000
27924,221LO0056200001,2.000000


In [44]:
# 기준일자가 월말인 DataFrame 필터링
monthly_dlqt = daily_dlqt[daily_dlqt['DLYSTADAT'].dt.is_month_end].reset_index()
# index 컬럼 지우기
monthly_dlqt.drop(['index'], axis=1, inplace=True)

In [45]:
monthly_dlqt

,DLYSTADAT,EXCFLDNBR,PAPCUSCDE,SCHFLDKND,SCHSEQNBR,RFDFLDODR,PMTAFTDAT,ACPPMTDAT,DLYTOTDAY,UNCPRNAMT,DLYPRNAMT,DLYINTAMT,DLYETCAMT,DLYLESAMT,CLNINTAMT,ACT_DLQT_DAY,DLQT_CLASS
0,2019-06-30,216AL1000270001,0,FI670001,31,30,2019-06-25,2019-06-25,5,80166407,1452755,0,0,3820,0,5,5D
1,2017-06-30,216AL1000410001,0,FI670001,7,6,2017-06-25,2017-06-26,5,170239293,2112303,994497,0,6128,0,4,U5
2,2017-08-31,216AL1000410001,0,FI670001,9,8,2017-08-25,2017-08-25,6,165978052,2136750,970050,0,10214,0,6,5D
3,2017-10-31,216AL1000410001,0,FI670001,11,10,2017-10-25,2017-10-25,6,161667493,2161480,945320,0,10214,0,6,5D
4,2017-12-31,216AL1000410001,0,FI670001,13,12,2017-12-25,2017-12-26,6,157307045,2186496,920304,0,8171,0,5,5D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,2021-06-30,221LO0043720001,0,FI670001,3,2,2021-06-28,2021-06-28,2,300000000,0,1043489,0,208,0,2,U5
12664,2021-06-30,221LO0044050001,0,FI670001,3,2,2021-06-29,2021-06-29,1,45000000,0,123950,0,24,0,1,U5
12665,2021-06-30,221LO0046820001,0,FI670001,2,1,2021-06-25,2021-06-25,5,170000000,0,183154,0,33,0,5,5D
12666,2021-06-30,221LO0048030001,0,FI670001,2,1,2021-06-25,2021-06-25,5,10000000,0,56910,0,18,0,5,5D


In [52]:
# 기준월 Label 작업
monthly_dlqt['MONTH_BASIS'] = monthly_dlqt['DLYSTADAT'].dt.strftime('%Y-%m')

In [53]:
# 누적 연체원금, 연체이자, 연체기타
monthly_dlqt['CUM_PRINCIPAL_DLQT'] = monthly_dlqt.groupby(by=['EXCFLDNBR'])['DLYPRNAMT'].transform(lambda x: x.cumsum())
monthly_dlqt['CUM_INTEREST_DLQT'] = monthly_dlqt.groupby(by=['EXCFLDNBR'])['DLYINTAMT'].transform(lambda x: x.cumsum())
monthly_dlqt['CUM_ETC_DLQT'] = monthly_dlqt.groupby(by=['EXCFLDNBR'])['DLYETCAMT'].transform(lambda x: x.cumsum())

In [54]:
monthly_dlqt

,DLYSTADAT,EXCFLDNBR,PAPCUSCDE,SCHFLDKND,SCHSEQNBR,RFDFLDODR,PMTAFTDAT,ACPPMTDAT,DLYTOTDAY,UNCPRNAMT,...,DLYINTAMT,DLYETCAMT,DLYLESAMT,CLNINTAMT,ACT_DLQT_DAY,DLQT_CLASS,CUM_PRINCIPAL_DLQT,CUM_INTEREST_DLQT,CUM_ETC_DLQT,MONTH_BASIS
0,2019-06-30,216AL1000270001,0,FI670001,31,30,2019-06-25,2019-06-25,5,80166407,...,0,0,3820,0,5,5D,1452755,0,0,2019-06
1,2017-06-30,216AL1000410001,0,FI670001,7,6,2017-06-25,2017-06-26,5,170239293,...,994497,0,6128,0,4,U5,2112303,994497,0,2017-06
2,2017-08-31,216AL1000410001,0,FI670001,9,8,2017-08-25,2017-08-25,6,165978052,...,970050,0,10214,0,6,5D,4249053,1964547,0,2017-08
3,2017-10-31,216AL1000410001,0,FI670001,11,10,2017-10-25,2017-10-25,6,161667493,...,945320,0,10214,0,6,5D,6410533,2909867,0,2017-10
4,2017-12-31,216AL1000410001,0,FI670001,13,12,2017-12-25,2017-12-26,6,157307045,...,920304,0,8171,0,5,5D,8597029,3830171,0,2017-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,2021-06-30,221LO0043720001,0,FI670001,3,2,2021-06-28,2021-06-28,2,300000000,...,1043489,0,208,0,2,U5,0,1919006,0,2021-06
12664,2021-06-30,221LO0044050001,0,FI670001,3,2,2021-06-29,2021-06-29,1,45000000,...,123950,0,24,0,1,U5,0,123950,0,2021-06
12665,2021-06-30,221LO0046820001,0,FI670001,2,1,2021-06-25,2021-06-25,5,170000000,...,183154,0,33,0,5,5D,0,183154,0,2021-06
12666,2021-06-30,221LO0048030001,0,FI670001,2,1,2021-06-25,2021-06-25,5,10000000,...,56910,0,18,0,5,5D,0,56910,0,2021-06
